In [40]:
import pandas as pd
import numpy as np
pd.options.plotting.backend = "plotly"

import plotly.express as px

In [41]:
fod = pd.read_csv('../data/Food/food-emissions-supply-chain.csv')
fod

,Entity,Code,Year,food_emissions_land_use,food_emissions_farm,food_emissions_animal_feed,food_emissions_processing,food_emissions_transport,food_emissions_retail,food_emissions_packaging,food_emissions_losses
0,Apples,NaN,2018,-0.028951,0.225816,0.000000,0.003820,0.095804,0.016578,0.044526,0.070808
1,Bananas,NaN,2018,-0.025534,0.269618,0.000000,0.060146,0.295830,0.020981,0.065670,0.175185
2,Barley,NaN,2018,0.008676,0.176377,0.000000,0.127703,0.035351,0.263587,0.496786,0.070692
3,Beef (beef herd),NaN,2018,23.237536,56.228063,2.680979,1.811083,0.494125,0.233538,0.352084,14.439998
4,Beef (dairy herd),NaN,2018,1.266022,21.915529,3.503673,1.547664,0.592410,0.254032,0.374549,3.847515
5,Beet Sugar,NaN,2018,0.000052,0.535721,0.000000,0.241885,0.628544,0.039322,0.090289,0.276263
6,Berries & Grapes,NaN,2018,0.024601,0.717848,0.000000,0.000000,0.236392,0.017057,0.211092,0.324895
7,Brassicas,NaN,2018,0.002242,0.277745,0.000000,0.000000,0.094641,0.016844,0.045303,0.077781
8,Cane Sugar,NaN,2018,1.263041,0.491270,0.000000,0.037381,0.794510,0.036723,0.084277,0.491722
9,Cassava,NaN,2018,0.589436,0.219994,0.000000,0.000000,0.093597,0.039322,0.044822,0.328504


In [42]:
fod.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43 entries, 0 to 42
Data columns (total 11 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   Entity                      43 non-null     object 
 1   Code                        0 non-null      float64
 2   Year                        43 non-null     int64  
 3   food_emissions_land_use     43 non-null     float64
 4   food_emissions_farm         43 non-null     float64
 5   food_emissions_animal_feed  43 non-null     float64
 6   food_emissions_processing   43 non-null     float64
 7   food_emissions_transport    43 non-null     float64
 8   food_emissions_retail       43 non-null     float64
 9   food_emissions_packaging    43 non-null     float64
 10  food_emissions_losses       43 non-null     float64
dtypes: float64(9), int64(1), object(1)
memory usage: 3.8+ KB


In [43]:
fod = fod.drop('Year', axis=1)

In [44]:
fod['Entity'].value_counts()

Entity
Apples               1
Rapeseed Oil         1
Other Fruit          1
Other Pulses         1
Other Vegetables     1
Palm Oil             1
Peas                 1
Pig Meat             1
Potatoes             1
Poultry Meat         1
Rice                 1
Olive Oil            1
Root Vegetables      1
Shrimps (farmed)     1
Soy milk             1
Soybean Oil          1
Sunflower Oil        1
Tofu                 1
Tomatoes             1
Wheat & Rye          1
Onions & Leeks       1
Oatmeal              1
Bananas              1
Cheese               1
Barley               1
Beef (beef herd)     1
Beef (dairy herd)    1
Beet Sugar           1
Berries & Grapes     1
Brassicas            1
Cane Sugar           1
Cassava              1
Citrus Fruit         1
Nuts                 1
Coffee               1
Dark Chocolate       1
Eggs                 1
Fish (farmed)        1
Groundnuts           1
Lamb & Mutton        1
Maize                1
Milk                 1
Wine                 1
Name

In [45]:
fod.loc[fod['Entity'] ==  'Lamb & Mutton', 'Entity'] = 'Lamb'
fod.loc[fod['Entity'] ==  'Beef (dairy herd)', 'Entity'] = 'Beef milk'
fod.loc[fod['Entity'] ==  'Beef (beef herd)', 'Entity'] = 'Beef meat'
fod.drop(index=19, axis=0).reset_index(drop=True)

,Entity,Code,food_emissions_land_use,food_emissions_farm,food_emissions_animal_feed,food_emissions_processing,food_emissions_transport,food_emissions_retail,food_emissions_packaging,food_emissions_losses
0,Apples,NaN,-0.028951,0.225816,0.000000,0.003820,0.095804,0.016578,0.044526,0.070808
1,Bananas,NaN,-0.025534,0.269618,0.000000,0.060146,0.295830,0.020981,0.065670,0.175185
2,Barley,NaN,0.008676,0.176377,0.000000,0.127703,0.035351,0.263587,0.496786,0.070692
3,Beef meat,NaN,23.237536,56.228063,2.680979,1.811083,0.494125,0.233538,0.352084,14.439998
4,Beef milk,NaN,1.266022,21.915529,3.503673,1.547664,0.592410,0.254032,0.374549,3.847515
5,Beet Sugar,NaN,0.000052,0.535721,0.000000,0.241885,0.628544,0.039322,0.090289,0.276263
6,Berries & Grapes,NaN,0.024601,0.717848,0.000000,0.000000,0.236392,0.017057,0.211092,0.324895
7,Brassicas,NaN,0.002242,0.277745,0.000000,0.000000,0.094641,0.016844,0.045303,0.077781
8,Cane Sugar,NaN,1.263041,0.491270,0.000000,0.037381,0.794510,0.036723,0.084277,0.491722
9,Cassava,NaN,0.589436,0.219994,0.000000,0.000000,0.093597,0.039322,0.044822,0.328504


In [46]:
fod['Code'] = [x for x in range(1, 44)]
fod

,Entity,Code,food_emissions_land_use,food_emissions_farm,food_emissions_animal_feed,food_emissions_processing,food_emissions_transport,food_emissions_retail,food_emissions_packaging,food_emissions_losses
0,Apples,1,-0.028951,0.225816,0.000000,0.003820,0.095804,0.016578,0.044526,0.070808
1,Bananas,2,-0.025534,0.269618,0.000000,0.060146,0.295830,0.020981,0.065670,0.175185
2,Barley,3,0.008676,0.176377,0.000000,0.127703,0.035351,0.263587,0.496786,0.070692
3,Beef meat,4,23.237536,56.228063,2.680979,1.811083,0.494125,0.233538,0.352084,14.439998
4,Beef milk,5,1.266022,21.915529,3.503673,1.547664,0.592410,0.254032,0.374549,3.847515
5,Beet Sugar,6,0.000052,0.535721,0.000000,0.241885,0.628544,0.039322,0.090289,0.276263
6,Berries & Grapes,7,0.024601,0.717848,0.000000,0.000000,0.236392,0.017057,0.211092,0.324895
7,Brassicas,8,0.002242,0.277745,0.000000,0.000000,0.094641,0.016844,0.045303,0.077781
8,Cane Sugar,9,1.263041,0.491270,0.000000,0.037381,0.794510,0.036723,0.084277,0.491722
9,Cassava,10,0.589436,0.219994,0.000000,0.000000,0.093597,0.039322,0.044822,0.328504


In [106]:
wat = pd.read_csv('..\data\Food\IME food waste report - data - volume of water for production.csv', header=1)
wat

,Foodstuff,Quantity,"Water consumption, litres"
0,Chocolate,1 kg,"17,196"
1,Beef,1 kg,"15,415"
2,Sheep Meat,1 kg,"10,412"
3,Pork,1 kg,"5,988"
4,Butter,1 kg,"5,553"
5,Chicken meat,1 kg,"4,325"
6,Cheese,1 kg,"3,178"
7,Olives,1 kg,"3,025"
8,Rice,1 kg,"2,497"
9,Cotton,1 @ 250g,"2,495"


In [ ]:
wat['Code'] = 0
watd = {'Dark chocolate': 17.196, 'Beef meat': 15.415, 'Lamb': 10.412, 'Pig Meat': 5.988, 'Poultry Meat': 4.325, 'Cheese': 3.178, 'Olive Oil': 3.025, 'Rice': 2.497, 'Wheat & Rye': 1.849, 'Apples': 822, 'Bananas': 790, 'Potatoes': 287, 'Beef milk': 1020, 'Tomatoes': 214, 'Eggs': 196, 'Wine': 416}

In [47]:
columns = ['Entity', 'Code', 'Year', 'Food', 'Feed', 'Processed']

soy2 = pd.read_csv('..\data\Soy\soybean-production-and-use.csv', names=columns, header=1)
soy2

,Entity,Code,Year,Food,Feed,Processed
0,Afghanistan,AFG,2015,0.0,0.0,NaN
1,Afghanistan,AFG,2016,0.0,0.0,NaN
2,Afghanistan,AFG,2017,0.0,0.0,NaN
3,Afghanistan,AFG,2018,0.0,0.0,NaN
4,Afghanistan,AFG,2019,0.0,3000.0,NaN
...,...,...,...,...,...,...
12077,Zimbabwe,ZWE,2016,2000.0,NaN,54000.0
12078,Zimbabwe,ZWE,2017,4000.0,NaN,82000.0
12079,Zimbabwe,ZWE,2018,5000.0,NaN,67000.0
12080,Zimbabwe,ZWE,2019,1000.0,NaN,66000.0


In [48]:
soy2['Entity'].value_counts()

Entity
Zimbabwe                         60
Malta                            60
Low-income countries             60
Lower-middle-income countries    60
Tanzania                         60
                                 ..
Bhutan                            2
Micronesia (country)              2
Bahrain                           2
Nauru                             2
South Sudan                       2
Name: count, Length: 239, dtype: int64

In [49]:
columns = ['Entity', 'Code', 'Year', 'Production', 'Yield', 'Area harvested']

soy3 = pd.read_csv('..\data\Soy\soy-production-yield-area.csv', names=columns, header=1)
soy3

,Entity,Code,Year,Production,Yield,Area harvested
0,Africa,NaN,1962,84594.0,0.393471,214994.0
1,Africa,NaN,1963,87260.0,0.394851,220995.0
2,Africa,NaN,1964,76781.0,0.369237,207945.0
3,Africa,NaN,1965,73067.0,0.347998,209964.0
4,Africa,NaN,1966,77180.0,0.374569,206050.0
...,...,...,...,...,...,...
7421,Zimbabwe,ZWE,2017,36478.0,1.551300,23515.0
7422,Zimbabwe,ZWE,2018,69688.0,1.868000,37307.0
7423,Zimbabwe,ZWE,2019,23043.0,0.695400,33136.0
7424,Zimbabwe,ZWE,2020,59656.0,1.814700,32874.0


In [50]:
# soy3.loc[soy3['Entity'].isin(['North America', 'Northern America (FAO)']), ['Entity', 'Year', 'Soybeans | 00000236 || Production | 005510 || tonnes', 'Soybeans | 00000236 || Yield | 005419 || tonnes per hectare']]

In [51]:
soy3 = soy3.drop(['Code'], axis=1)


In [52]:
soy = pd.merge(soy2, soy3, on=['Entity', 'Year']).reset_index(drop=True)
soy.shape

(7104, 9)

In [53]:
soy = soy[~soy['Entity'].str.contains('(FAO)')].reset_index(drop=True)
soy['Entity'].value_counts()


C:\Users\Nerea\AppData\Local\Temp\ipykernel_8444\4281398641.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  soy = soy[~soy['Entity'].str.contains('(FAO)')]


Entity
Liberia        60
Mexico         60
Pakistan       60
Oceania        60
North Korea    60
               ..
Malta           3
Bhutan          2
Cyprus          2
Luxembourg      1
Finland         1
Name: count, Length: 131, dtype: int64

In [54]:
lst = soy['Entity'].value_counts().index.tolist()
lst

['Liberia',
 'Mexico',
 'Pakistan',
 'Oceania',
 'North Korea',
 'North America',
 'Nigeria',
 'Nepal',
 'Myanmar',
 'Malaysia',
 'Peru',
 'Lower-middle-income countries',
 'Low-income countries',
 'Laos',
 'Japan',
 'Italy',
 'Iran',
 'Indonesia',
 'Paraguay',
 'Philippines',
 'High-income countries',
 'Timor',
 'World',
 'Vietnam',
 'Uruguay',
 'Upper-middle-income countries',
 'United States',
 'Uganda',
 'Turkey',
 'Thailand',
 'Romania',
 'Tanzania',
 'Taiwan',
 'Suriname',
 'Sri Lanka',
 'South Korea',
 'South America',
 'South Africa',
 'India',
 'Zimbabwe',
 'Cambodia',
 'Colombia',
 'Argentina',
 'Asia',
 'Australia',
 'China',
 'Brazil',
 'Canada',
 'European Union (27)',
 'Bulgaria',
 'Europe',
 'Ecuador',
 'Africa',
 'Hungary',
 'Rwanda',
 'Bolivia',
 'Chile',
 'El Salvador',
 'Spain',
 'Egypt',
 'Zambia',
 'France',
 'Iraq',
 'Nicaragua',
 'Madagascar',
 'New Zealand',
 "Cote d'Ivoire",
 'Cameroon',
 'Albania',
 'Mali',
 'Guatemala',
 'Morocco',
 'Burkina Faso',
 'Costa Ri

In [55]:
soy.loc[soy['Entity'] == 'European Union (27)', 'Entity'] = 'European Union'
soy.loc[soy['Entity'] =="Cote d'Ivoire", 'Entity'] = 'Ivory Coast'
soy.loc[soy['Entity'] == 'Democratic Republic of Congo'] = 'Congo'

In [56]:
soy.head(10)

,Entity,Code,Year,Food,Feed,Processed,Production,Yield,Area harvested
0,Africa,NaN,1962,44000.0,7000.0,17000.0,84594.0,0.393471,214994.0
1,Africa,NaN,1963,31000.0,7000.0,5000.0,87260.0,0.394851,220995.0
2,Africa,NaN,1964,43000.0,6000.0,14000.0,76781.0,0.369237,207945.0
3,Africa,NaN,1965,34000.0,6000.0,12000.0,73067.0,0.347998,209964.0
4,Africa,NaN,1966,41000.0,6000.0,2000.0,77180.0,0.374569,206050.0
5,Africa,NaN,1967,48000.0,6000.0,4000.0,80884.0,0.418871,193100.0
6,Africa,NaN,1968,49000.0,7000.0,3000.0,90459.0,0.416296,217295.0
7,Africa,NaN,1969,52000.0,6000.0,6000.0,89000.0,0.459376,193741.0
8,Africa,NaN,1970,53000.0,6000.0,8000.0,97211.0,0.436648,222630.0
9,Africa,NaN,1971,60000.0,6000.0,2000.0,97545.0,0.412102,236701.0


In [57]:
soy['Code'] = pd.factorize(soy['Entity'])[0]+1
soy.head()

,Entity,Code,Year,Food,Feed,Processed,Production,Yield,Area harvested
0,Africa,1,1962,44000.0,7000.0,17000.0,84594.0,0.393471,214994.0
1,Africa,1,1963,31000.0,7000.0,5000.0,87260.0,0.394851,220995.0
2,Africa,1,1964,43000.0,6000.0,14000.0,76781.0,0.369237,207945.0
3,Africa,1,1965,34000.0,6000.0,12000.0,73067.0,0.347998,209964.0
4,Africa,1,1966,41000.0,6000.0,2000.0,77180.0,0.374569,206050.0


In [58]:
len(soy['Entity'].value_counts())

131

In [59]:
len(soy['Code'].value_counts())

131

In [68]:
columns = ['Entity', 'Code', 'Year', 'Coconut', 'Groundnut', 'Palm', 'Sesame', 'Sunflower', 'Cottonseed', 'Olive', 'Rapeseed', 'Soybean']

oil_yield = pd.read_csv('..\data\Palm\oil-yield-by-crop.csv', names=columns, header=1)
oil_yield

,Entity,Code,Year,Coconut,Groundnut,Palm,Sesame,Sunflower,Cottonseed,Olive,Rapeseed,Soybean
0,Afghanistan,AFG,1962,NaN,NaN,NaN,0.056506,0.378042,0.084737,0.1496,NaN,NaN
1,Afghanistan,AFG,1963,NaN,NaN,NaN,0.055145,0.378042,0.096713,0.1360,NaN,NaN
2,Afghanistan,AFG,1964,NaN,NaN,NaN,0.082000,0.378042,0.065567,0.1496,NaN,NaN
3,Afghanistan,AFG,1965,NaN,NaN,NaN,0.084964,0.390090,0.097932,0.1360,NaN,NaN
4,Afghanistan,AFG,1966,NaN,NaN,NaN,0.094843,0.414187,0.134445,0.1496,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
12162,Zimbabwe,ZWE,2016,NaN,0.037665,NaN,NaN,0.166113,0.061971,NaN,NaN,0.240391
12163,Zimbabwe,ZWE,2017,NaN,0.036669,NaN,NaN,0.181400,0.082358,NaN,NaN,0.612375
12164,Zimbabwe,ZWE,2018,NaN,0.049197,NaN,NaN,0.170973,0.074162,NaN,NaN,0.394028
12165,Zimbabwe,ZWE,2019,NaN,0.060487,NaN,NaN,0.097837,0.110400,NaN,NaN,0.250483


In [70]:
columns = ['Entity', 'Code', 'Year', 'Sesame', 'Maize', 'Safflower', 'Coconut', 'Sunflower', 'Rapeseed', 'Palm kernel', 'Groundnut', 'Linseed', 'Olive', 'Palm', 'Soybean', 'Cottonseed']

oil_prod = pd.read_csv(r"..\data\Palm\vegetable-oil-production.csv", names=columns, header=1)
oil_prod

,Entity,Code,Year,Sesame,Maize,Safflower,Coconut,Sunflower,Rapeseed,Palm kernel,Groundnut,Linseed,Olive,Palm,Soybean,Cottonseed
0,Afghanistan,AFG,2011,1292.0,NaN,NaN,NaN,60.0,NaN,NaN,NaN,674.65,538.56,NaN,NaN,3725.00
1,Afghanistan,AFG,2012,1261.0,NaN,NaN,NaN,61.0,NaN,NaN,NaN,706.30,538.56,NaN,NaN,4097.97
2,Afghanistan,AFG,2013,1334.0,NaN,NaN,NaN,138.0,NaN,NaN,NaN,733.57,545.93,NaN,NaN,4761.86
3,Afghanistan,AFG,2014,1250.0,NaN,NaN,NaN,960.0,NaN,NaN,NaN,790.00,658.42,NaN,NaN,4832.28
4,Afghanistan,AFG,2015,861.0,NaN,NaN,NaN,1035.0,NaN,NaN,NaN,5374.63,711.37,NaN,NaN,5690.84
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12703,Zimbabwe,ZWE,1964,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7560.0,NaN,NaN,NaN,NaN,540.00
12704,Zimbabwe,ZWE,1965,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6300.0,NaN,NaN,NaN,NaN,1260.00
12705,Zimbabwe,ZWE,1966,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4956.0,NaN,NaN,NaN,NaN,1260.00
12706,Zimbabwe,ZWE,1967,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7518.0,NaN,NaN,NaN,NaN,3474.00


In [69]:
oil_yield.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12167 entries, 0 to 12166
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Entity      12167 non-null  object 
 1   Code        9589 non-null   object 
 2   Year        12167 non-null  int64  
 3   Coconut     5396 non-null   float64
 4   Groundnut   6774 non-null   float64
 5   Palm        3584 non-null   float64
 6   Sesame      4345 non-null   float64
 7   Sunflower   5470 non-null   float64
 8   Cottonseed  7127 non-null   float64
 9   Olive       3369 non-null   float64
 10  Rapeseed    4762 non-null   float64
 11  Soybean     5594 non-null   float64
dtypes: float64(9), int64(1), object(2)
memory usage: 1.1+ MB


In [71]:
oil_yield = oil_yield[~oil_yield['Entity'].str.contains('(FAO)')].reset_index(drop=True)
oil_yield.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10300 entries, 0 to 12166
Data columns (total 12 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Entity      10300 non-null  object 
 1   Code        9589 non-null   object 
 2   Year        10300 non-null  int64  
 3   Coconut     4201 non-null   float64
 4   Groundnut   5139 non-null   float64
 5   Palm        2590 non-null   float64
 6   Sesame      3027 non-null   float64
 7   Sunflower   3949 non-null   float64
 8   Cottonseed  5478 non-null   float64
 9   Olive       2288 non-null   float64
 10  Rapeseed    3342 non-null   float64
 11  Soybean     4026 non-null   float64
dtypes: float64(9), int64(1), object(2)
memory usage: 1.0+ MB


C:\Users\Nerea\AppData\Local\Temp\ipykernel_8444\2273293486.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  oil_yield = oil_yield[~oil_yield['Entity'].str.contains('(FAO)')]


In [ ]:
oil_yield.loc[oil_yield['Entity'] == 'European Union (27)', 'Entity'] = 'European Union'
oil_yield.loc[oil_yield['Entity'] =="Cote d'Ivoire", 'Entity'] = 'Ivory Coast'
oil_yield.loc[oil_yield['Entity'] == 'Democratic Republic of Congo'] = 'Congo'

In [72]:
oil_prod = oil_prod[~oil_prod['Entity'].str.contains('(FAO)')].reset_index(drop=True)
oil_prod.info()

<class 'pandas.core.frame.DataFrame'>
Index: 10841 entries, 0 to 12707
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Entity       10841 non-null  object 
 1   Code         10130 non-null  object 
 2   Year         10841 non-null  int64  
 3   Sesame       739 non-null    float64
 4   Maize        3419 non-null   float64
 5   Safflower    320 non-null    float64
 6   Coconut      5869 non-null   float64
 7   Sunflower    988 non-null    float64
 8   Rapeseed     819 non-null    float64
 9   Palm kernel  3785 non-null   float64
 10  Groundnut    6065 non-null   float64
 11  Linseed      4061 non-null   float64
 12  Olive        2485 non-null   float64
 13  Palm         3153 non-null   float64
 14  Soybean      1208 non-null   float64
 15  Cottonseed   6166 non-null   float64
dtypes: float64(13), int64(1), object(2)
memory usage: 1.4+ MB


C:\Users\Nerea\AppData\Local\Temp\ipykernel_8444\3639632926.py:1: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  oil_prod = oil_prod[~oil_prod['Entity'].str.contains('(FAO)')]


In [75]:
oil_prod.loc[oil_prod['Entity'] == 'European Union (27)', 'Entity'] = 'European Union'
oil_prod.loc[oil_prod['Entity'] =="Cote d'Ivoire", 'Entity'] = 'Ivory Coast'
oil_prod.loc[oil_prod['Entity'] == 'Democratic Republic of Congo'] = 'Congo'
oil_prod.loc[oil_prod['Entity'] == 'Micronesia (country)'] = 'Micronesia'


In [ ]:
# Dejamos los valores nulos porque representan que no existe la producción de ese vegetale en ese momento. Rellenarlos con valores sería sesgar la info.

In [89]:
# Creamos un diccionario con los nombres de paises y su id. Se aplica mapeando a los dos df de oil.

dic = soy.set_index('Entity')['Code'].to_dict()
oil_prod['Code'] = oil_prod['Entity'].map(dic)
oil_yield['Code'] = oil_yield['Entity'].map(dic)

C:\Users\Nerea\AppData\Local\Temp\ipykernel_8444\263353594.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oil_prod['Code'] = oil_prod['Entity'].map(dic)
C:\Users\Nerea\AppData\Local\Temp\ipykernel_8444\263353594.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  oil_yield['Code'] = oil_yield['Entity'].map(dic)


In [96]:
max(soy['Code'])

131

In [103]:
nul_ind = oil_prod['Code'].isnull()
fac = pd.factorize(nul_ind)[0] + 132
ser = pd.Series(fac, index=nul_ind.index)

oil_prod['Code'] = oil_prod['Code'].fillna(ser)
oil_prod['Code'].astype(int)


0        132
1        132
2        132
3        132
4        132
        ... 
10836    131
10837    131
10838    131
10839    131
10840    131
Name: Code, Length: 10841, dtype: int32

In [102]:
oil_prod['Code'].isnull().any()

False

In [104]:
oil_prod['Code'].value_counts()

Code
132.0    3937
31.0      120
79.0       60
91.0       60
90.0       60
         ... 
101.0      14
44.0       11
14.0       11
65.0       11
10.0       11
Name: count, Length: 131, dtype: int64